In [1]:
import os
#import pandas as pd
import numpy as np
import cv2
#from PIL import Image
import random
#import tensorflow as tf
#import re
#import datetime
#import io
#from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
#import pickle
#from sklearn.model_selection import train_test_split
from nltk.metrics.distance import edit_distance
#import string
#from utils import generate_token_index
#from utils import score_prediction, generate_token_index, y_labels, generate_dataset
import json
import keras

%matplotlib inline

Using TensorFlow backend.


### Load filenames and labels

In [2]:
data_folder = '../datasets/'

In [3]:
with open(data_folder + 'gt.json') as f:
    dataset = json.load(f)

In [4]:
dataset_train_orig = dataset['train']
dataset_val = dataset['val']
dataset_test = dataset['test']

### Select only a subsample for training

In [5]:
#select only a fraction
fraction = 0.1
max_len = int(len(dataset_train_orig) * fraction)

indices = np.random.randint(0, len(dataset_train_orig), max_len)
dataset_train = [dataset_train_orig[j] for j in indices]
#len(dataset)
#dataset = dataset_orig
print(len(dataset_train))

9340


In [6]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, dataset, y_size, x_size, n_channels, batch_size = 16, shuffle=True):
        self.dataset = dataset
        self.dataset_len = len(dataset)
        self.batch_size = batch_size
        #self.labels = to_categorical(labels)
        #self.filenames = filenames
        self.y_size = y_size
        self.x_size = x_size
        self.n_channels = n_channels
        self.shuffle = shuffle

        self.max_decoder_seq_length, self.decoder_tokens = self.generate_decoder_tokens()
        self.target_token_index, self.num_decoder_tokens = self.generate_token_index()
        
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(self.dataset_len / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        dataset_temp = [self.dataset[k] for k in indexes]
#        labels_temp = [self.labels[k] for k in indexes]
        
        # Generate data
        X, y1, y2 = self.data_generation(dataset_temp)

        return [X, y1], y2

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.dataset_len)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def data_generation(self, dataset_temp):        
        batch_x = []
        batch_y1 = []
        batch_y2 = []

        for elem in dataset_temp:
            #img = cv2.imread(filename, 0)
            #resized_image = cv2.resize(img, (self.x_size, self.y_size)) 
            #normalized_image = resized_image / 255.
            
            normalized_image = self.preprocess_image(elem[0])
            decoder_input_data, decoder_target_data = self.one_hot_labels(elem[1], self.max_decoder_seq_length, 
                                                                           self.num_decoder_tokens, 
                                                                           self.target_token_index)

            batch_x.append(normalized_image)

            batch_y1.append(decoder_input_data)
            batch_y2.append(decoder_target_data)
        
        batch_x = np.asarray(batch_x, dtype = np.float32)

#        batch_x = np.asarray(np.reshape(batch_x, (-1, self.y_size, self.x_size, self.n_channels)), dtype = np.float32)
        batch_y1 = np.asarray(batch_y1, dtype = np.float32)
        batch_y2 = np.asarray(batch_y2, dtype = np.float32)
        
        return batch_x, batch_y1, batch_y2
    
    def preprocess_image(self, filename):
#        imgs = []
#        words = []

#        for ind in range(len(dataset)):

        #for ind in range(30):
        #    print(ind)
#            fpath = dataset[ind][1]
        #print(file)
        img = cv2.imread(filename, 0)
        
        if img is not None:            
            (wt, ht) = self.x_size, self.y_size
            (h, w) = img.shape
            fx = w / wt
            fy = h / ht
            f = max(fx, fy)
            newSize = (max(min(wt, int(w / f)), 1), max(min(ht, int(h / f)), 1)) # scale according to f (result at least 1 and at most wt or ht)
            img = cv2.resize(img, newSize)
            target = np.ones([ht, wt]) * 255
            target[0:newSize[1], 0:newSize[0]] = img

            # transpose for TF
            #img = cv2.transpose(target)

            img = target

            # normalize
            (m, s) = cv2.meanStdDev(img)
            m = m[0][0]
            s = s[0][0]
            img = img - m
            img = img / s if s>0 else img            

            img = np.reshape(img, (self.y_size, self.x_size, self.n_channels))
            #imgs.append(img)
            #words.append(dataset[ind][0])

#        array_images = np.asarray(imgs)

#        return array_images, words 
        return img

    def generate_decoder_tokens(self):
        #target_texts = []
        max_decoder_seq_length = 0.
        decoder_tokens = set()

        for elem in self.dataset:
            word = elem[1]
        #    word = '[' + word + ']'
            decoder_tokens.add('[')
            decoder_tokens.add(']')

            if len(word) > max_decoder_seq_length:
                max_decoder_seq_length = len(word)
            for letter in word:
                decoder_tokens.add(letter)
            #    input_text, target_text = line.split('\t')
            # We use "tab" as the "start sequence" character
            # for the targets, and "\n" as "end sequence" character.

        #    input_texts.append(input_text)
        #    target_texts.append(word

        max_decoder_seq_length += 2    
        decoder_tokens  = list(decoder_tokens)
        
        return max_decoder_seq_length, decoder_tokens
    
    def generate_token_index(self):
    #decoder_tokens += ':[]'
        num_decoder_tokens = len(self.decoder_tokens)

        return dict((k, v) for v, k in enumerate(self.decoder_tokens)), num_decoder_tokens
    
    def one_hot_labels(self, label, max_decoder_seq_length, num_decoder_tokens, target_token_index):

#        decoder_input_data = np.zeros( (len(ys),  max_decoder_seq_length, num_decoder_tokens), dtype='float32')
#        decoder_target_data = np.zeros( (len(ys), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

        decoder_input_data = np.zeros((max_decoder_seq_length, num_decoder_tokens), dtype='float32')
        decoder_target_data = np.zeros((max_decoder_seq_length, num_decoder_tokens), dtype='float32')
        
#        for i, target_text in enumerate(ys):
            
        label = '[' + label + ']'

        for t, char in enumerate(label):
            decoder_input_data[t, target_token_index[char]] = 1.
            if t > 0:
                decoder_target_data[t - 1, target_token_index[char]] = 1.

        return decoder_input_data, decoder_target_data    

In [7]:
train_generator = DataGenerator(dataset_train, 128, 384, 1, batch_size = 64)
val_generator = DataGenerator(dataset_val, 128, 384, 1, batch_size = 64)

#[batch_x, batch_y1], batch_y2 = generator.__getitem__(0)

In [9]:
y_size = train_generator.y_size
x_size = train_generator.x_size
max_decoder_seq_length = train_generator.max_decoder_seq_length
num_decoder_tokens = train_generator.num_decoder_tokens

In [ ]:
#batch_x.shape

In [ ]:
#len(batch_x[0])

In [ ]:
#index = random.randint(0, array_images.shape[0] - 1)
plt.figure(figsize=(9, 9)) 
#plt.imshow(np.reshape(array_images[index, :, :], (y_size, x_size)), cmap=plt.get_cmap('gray'))
plt.imshow(batch_x[0], cmap=plt.get_cmap('gray'))
plt.show()
#print("word = ", words[index])

### Keras model

In [10]:
#from __future__ import print_function
from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense, TimeDistributed, Conv2D, MaxPooling2D, Reshape, Dropout, BatchNormalization, Activation, Bidirectional, concatenate, add, Lambda, Permute
from keras.callbacks import EarlyStopping
#import keras.backend as K
from keras.optimizers import Adam

In [11]:
latent_dim = 512  # Latent dimensionality of the encoding space.
#optimizer
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [12]:
## Define an input sequence and process it.
encoder_inputs = Input(shape=(y_size, x_size, 1), name='input_encoder')

#encoder_inputs = Input(shape=(128, 384, 1), name='input_encoder')

#encoder_inputs = Input(shape=(92, 248, 1), name='input_encoder')

#(7,7)
encoder_layer = Conv2D(16, (7, 7), padding='same', use_bias=False, name='conv_1')(encoder_inputs)
encoder_layer = BatchNormalization(name='batch_norm_1')(encoder_layer)
encoder_layer = Activation('relu', name='activation_1')(encoder_layer)
encoder_layer = MaxPooling2D(pool_size=(2, 2), padding='valid', name='maxpool_1')(encoder_layer)

#(7,7)
encoder_layer = Conv2D(32, (5, 5), padding='same', use_bias=False, name='conv_2')(encoder_layer)
encoder_layer = BatchNormalization(name='batch_norm_2')(encoder_layer)
encoder_layer = Activation('relu', name='activation_2')(encoder_layer)
encoder_layer = MaxPooling2D(pool_size=(2, 2), padding='valid', name='maxpool_2')(encoder_layer)

#(5,5)
encoder_layer = Conv2D(64, (5, 5), padding='same', use_bias=False, name='conv_3')(encoder_layer)
encoder_layer = BatchNormalization(name='batch_norm_3')(encoder_layer)
encoder_layer = Activation('relu', name='activation_3')(encoder_layer)
encoder_layer = MaxPooling2D(pool_size=(2, 2), padding='valid', name='maxpool_3')(encoder_layer)

#(5,5)
encoder_layer = Conv2D(128, (3, 3), padding='same', use_bias=False, name='conv_4')(encoder_layer)
encoder_layer = BatchNormalization(name='batch_norm_4')(encoder_layer)
encoder_layer = Activation('relu', name='activation_4')(encoder_layer)
encoder_layer = MaxPooling2D(pool_size=(2, 2), padding='valid', name='maxpool_4')(encoder_layer)

#(3,3)
encoder_layer = Conv2D(256, (3, 3), padding='same', use_bias=False, name='conv_5')(encoder_layer)
encoder_layer = BatchNormalization(name='batch_norm_5')(encoder_layer)
encoder_layer = Activation('relu', name='activation_5')(encoder_layer)
encoder_layer = MaxPooling2D(pool_size=(2, 2), padding='valid', name='maxpool_5')(encoder_layer)

#(3,3)
encoder_layer = Conv2D(512, (3, 3), padding='same', use_bias=False, name='conv_6')(encoder_layer)
encoder_layer = BatchNormalization(name='batch_norm_6')(encoder_layer)
encoder_layer = Activation('relu', name='activation_6')(encoder_layer)
encoder_layer = MaxPooling2D(pool_size=(2, 2), padding='valid', name='maxpool_6')(encoder_layer)


#(2,2)
#encoder_layer = Conv2D(512, (3, 3), padding='same', use_bias=False, name='conv_6')(encoder_layer)
#encoder_layer = BatchNormalization(name='batch_norm_6')(encoder_layer)
#encoder_layer = Activation('relu', name='activation_6')(encoder_layer)
#encoder_layer = MaxPooling2D(pool_size=(2, 2), padding='valid', name='maxpool_6')(encoder_layer)


#(3,3)
#encoder_layer = Conv2D(512, (3, 3), padding='same', use_bias=False, name='conv_6')(encoder_layer)
#encoder_layer = BatchNormalization(name='batch_norm_6')(encoder_layer)
#encoder_layer = Activation('relu', name='activation_6')(encoder_layer)
#encoder_layer = MaxPooling2D(pool_size=(2, 2), padding='valid', name='maxpool_6')(encoder_layer)

conv_shapes = encoder_layer.shape[1:]
timesteps = int(conv_shapes[0]*conv_shapes[1])
num_features = int(conv_shapes[2])

encoder_layer = Reshape((-1, num_features), name='reshape')(encoder_layer)

#encoder
encoder = LSTM(latent_dim, return_state=True, name='lstm_encoder')
_, state_h, state_c = encoder(encoder_layer)
encoder_states = [state_h, state_c]

#decoder
decoder_inputs = Input(shape=(max_decoder_seq_length, num_decoder_tokens), name='input_decoder_teacher_forcing')
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='lstm_decoder')
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_dense_time_dist = TimeDistributed(Dense(num_decoder_tokens, activation='softmax'), name='time_distributed_dense')
decoder_outputs = decoder_dense_time_dist(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_encoder (InputLayer)      (None, 128, 384, 1)  0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 128, 384, 16) 784         input_encoder[0][0]              
__________________________________________________________________________________________________
batch_norm_1 (BatchNormalizatio (None, 128, 384, 16) 64          conv_1[0][0]                     
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 128, 384, 16) 0           batch_norm_1[0][0]               
__________________________________________________________________________________________________
maxpool_1 

In [15]:
train = True

#batch_size = 128  # Batch size for training.
epochs = 50 # Number of epochs to train for.

#n_times_aug = 1

# Early stopping  
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')

if train == True:    
    # Run training
    model.compile(optimizer=optimizer, loss='categorical_crossentropy')

#    model.fit([array_images_train, decoder_input_data_train], decoder_target_data_train, batch_size=batch_size,
#              epochs=epochs, validation_data = ([array_images_val, decoder_input_data_val], decoder_target_data_val),
#              verbose=1, callbacks=[])

    #model.fit_generator(generator = generator epochs=epochs, verbose=1)

#    model.fit_generator(generator=train_generator, validation_data=val_generator, epochs=epochs, verbose=1, 
#                        callbacks=[])

    model.fit_generator(generator=train_generator, epochs=epochs, verbose=1, 
                        callbacks=[])

    
#    model.fit_generator(train_datagen.flow([array_images_train, decoder_input_data_train], decoder_target_data_train, batch_size=batch_size),
#                        steps_per_epoch = n_times_aug * len(array_images_train) / batch_size, epochs=epochs, 
#                        validation_data = val_datagen.flow([array_images_val, decoder_input_data_val], decoder_target_data_val, batch_size=batch_size),
#                        validation_steps =  1, verbose=1, callbacks=[early_stop])

Epoch 1/50
145/145 [==============================] - 41s 283ms/step - loss: 0.4774
Epoch 2/50
145/145 [==============================] - 39s 270ms/step - loss: 0.4094
Epoch 3/50
145/145 [==============================] - 39s 269ms/step - loss: 0.3669
Epoch 4/50
145/145 [==============================] - 39s 269ms/step - loss: 0.3329
Epoch 5/50
145/145 [==============================] - 39s 269ms/step - loss: 0.3029
Epoch 6/50
145/145 [==============================] - 39s 271ms/step - loss: 0.2733
Epoch 7/50
145/145 [==============================] - 39s 268ms/step - loss: 0.2461
Epoch 8/50
145/145 [==============================] - 39s 269ms/step - loss: 0.2216
Epoch 9/50
145/145 [==============================] - 39s 269ms/step - loss: 0.1972
Epoch 10/50
145/145 [==============================] - 39s 266ms/step - loss: 0.1751
Epoch 11/50
126/145 [=========================>....] - ETA: 5s - loss: 0.1520

KeyboardInterrupt: 

In [ ]:
model_json = model.to_json()
with open("../snapshots/graph.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("../snapshots/weights.h5")

In [ ]:
#if train == False:
#model.save('../snapshots/model_IAM.h5')
#model.save_weights("../snapshots/weights.h5")

In [ ]:
if train == False:
    from keras.models import load_model, model_from_json
    json_file = open('../snapshots/graph.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("../snapshots/weights.h5")    
#    model = load_model('../snapshots/model_IAM.h5')

In [ ]:
#model.save_weights("../snapshots/weights.h5")

In [ ]:
# Define inference models 
#encoder
#encoder_inputs_inference = Input(shape=(x_size, y_size, 1), name='input_encoder_inference')
encoder_inference = Model(model.get_input_at(0)[0], model.get_layer("lstm_encoder").output[1:])

#decoder
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_inputs_inference = Input(shape=(1, num_decoder_tokens), name='input_decoder_inference')

decoder_lstm = model.get_layer("lstm_decoder")
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs_inference, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]

#dense layer
decoder_dense = model.get_layer('time_distributed_dense').layer
decoder_outputs = decoder_dense(decoder_outputs)

#inference
decoder_inference = Model([decoder_inputs_inference] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [ ]:
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq, max_decoder_seq_length=5):
    # Encode the input as state vectors.
    states_value = encoder_inference.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['[']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_inference.predict( [target_seq] + states_value )
           
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == ']'): 
            #or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
pred_train = []

for ind in range(len(words_train)):    
    #row_pred = []

    input_seq = np.reshape( array_images_train[ind,:,:], (-1, y_size,  x_size, 1) )
    
    decoded_sentence = decode_sequence(input_seq)
    decoded_sentence = decoded_sentence.replace("]", "")
    
    #row_pred.append(words_train[ind])
    #row_pred.append(decoded_sentence)
    
    pred_train.append(decoded_sentence)
    
    #print('Data', words_test[ind], '-', ind, 'out of', len(words_test))
    print('True sequence:', words_train[ind])
    print('Decoded sequence:', decoded_sentence, '\n')

In [ ]:
pred_test = []

for ind in range(len(words_test)):    
    #row_pred = []

    input_seq = np.reshape( array_images_test[ind,:,:], (-1, y_size,  x_size, 1) )
    
    decoded_sentence = decode_sequence(input_seq)
    decoded_sentence = decoded_sentence.replace("]", "")
    
    #row_pred.append(words_test[ind])
    #row_pred.append(decoded_sentence)
    
    pred_test.append(decoded_sentence)
    
    #print('Data', words_test[ind], '-', ind, 'out of', len(words_test))
    print('True sentence:', words_test[ind])
    print('Decoded sentence:', decoded_sentence, '\n')

In [ ]:
def score_prediction(y_true, y_pred):
    words_identified = 0
    characters_identified = 0
    char_tot = 0
#    CER = 0

#    list_accuracy_characters = []
    
    for i in range(len(y_pred)):
        #pred_row = [y_true[i], y_pred[i]] 
        
        #check if date are the same
        #if pred_row[0] == pred_row[1]:
        if y_true[i] == y_pred[i]:

            words_identified += 1
            
#        if len(pred_row[1]) < len(pred_row[0]):
#            pred_row[1] += '-' * (len(pred_row[0]) - len(pred_row[1]))    
#        elif len(pred_row[1]) > len(pred_row[1]):

#            pred_row[1] = pred_row[1][0:len(pred_row[0])]

        #check the number of characters that are the same
 #       print(y_true[i])
 #       print(y_pred[i])
        
        levenshtein_distance = edit_distance(y_true[i], y_pred[i])
        n_char = np.maximum(len(y_true[i]), len(y_pred[i]))
        
        normalized_distance = levenshtein_distance/n_char

        characters_identified += normalized_distance
#        char_tot += n_char
        
#        CER += normalized_distance
        
#        print(len(y_true[i]))
#        for k in range(len(y_true[i])):
#            print()
#            if y_true[i][k] == y_pred[1][k]:
#        characters_identified += 1
#        char_tot += 1

    # array_accuracy_characters = np.asarray(list_accuracy_characters)
    CER = float((characters_identified) / len(y_true))
    WER = (len(y_pred) - words_identified)/len(y_pred) 
        
    return CER, WER
#    return WER

In [ ]:
CER_test, WER_test = score_prediction(words_test, pred_test)
#WER_test = score_prediction(words_test, pred_test)
print('CER: ', round(CER_test * 100, 3), '%')
print('WER: ', round(WER_test * 100, 3), '%')

In [ ]:
CER_train, WER_train = score_prediction(words_train, pred_train)
#print('CER: ', round(CER_train * 100, 3), '%')
print('CER: ', round(CER_train * 100, 3), '%')
print('WER: ', round(WER_train * 100, 3), '%')